**Exercise set 5**
==============


>The goal of this exercise is to perform and analyze experimental
>designs. After this exercise you should have gained familiarity
>with both full and fractional factorial designs and concepts
>such as **effects**, **confounding**, **defining contrast**,
>**generators** and **resolution**.


**Exercise 5.1**

The growth rate of a certain bacterium specie depends
on the concentration of nutrients such as phosphate,
sucrose, and nitrate. We have conducted a set of
experiments where we have investigated how the
growth is influenced by the concentration of
phosphate ($P$), the concentration of sucrose ($S$) and
the concentration of nitrate ($N$). The design
matrix and the measured growth rate are given in table 1.


|$P$  | $S$ | $N$ | **Growth rate**  |
|:---:|:---:|:---:|:---:|
|$+$ | $-$ | $-$ | $7$ |
|$-$ | $+$ | $-$ | $10$ |
|$+$ | $-$ | $+$ | $8$ |
|$-$ | $+$ | $+$ | $11$ |
|$-$ | $-$ | $-$ | $11$ |
|$+$ | $+$ | $+$ | $12$ |
|$+$ | $+$ | $-$ | $7$ |
|$-$ | $-$ | $+$ | $7$ |

| |
|:---|
|**Table 1:** *Experimental design matrix for the growth rate of the investigated bacteria. The factors are the concentration of phosphate ($P$), the concentration of sucrose ($S$), and the concentration of nitrate ($N$).*|


**(a)**  Compute all the main effects.



In [1]:
# Here we code a plus as a 1, and - as a 0 (for easy manipulation later on)
data = {(1, 0, 0): 7,
        (0, 1, 0): 10,
        (1, 0, 1): 8,
        (0, 1, 1): 11,
        (0, 0, 0): 11,
        (1, 1, 1): 12,
        (1, 1, 0): 7,
        (0, 0, 1): 7,}

effects = ['P', 'S', 'N']
power = len(effects)-1
for i, e in enumerate(effects):
    eff = 0
    for key, value in data.items():
        if key[i]:
            eff += value
        else:
            eff -= value
    eff /= 2**power 
    print(f"Effect({e}): {eff}")

Effect(P): -1.25
Effect(S): 1.75
Effect(N): 0.75


**Answer to question 5.1(a):** 

The effects are calulated with the formula:

\begin{equation}
    \text{Effect}(X) = \frac{
        \sum_{i \in \text{high level runs}} y_i -
        \sum_{i \in \text{low level runs}} y_i
    }{2^{N-1}
    } .
\label{eq:effect}\tag{1}
\end{equation}

where $N$ is the number of variables considered (3 in our case) and printed above.

**(b)**  Extend the design matrix with the possible $2$-factor and $3$-factor
interaction effects. Compute these interaction effects.



In [2]:
# Add interaction effects
def new_key(key):
    # Use bitwise combinations
    ab = int()
    return (*key, 
            int(key[0] == key[1]),
            int(key[0] == key[2]),
            int(key[1] == key[2]),
            int(key[0] == (key[1] == key[2]))
           )

data2 = {new_key(key): value for key, value in data.items()}

effects.extend(['PS', 'PN', "SN", "PSN"])
power = 2 # Still only considering 3 variables
for i, e in enumerate(effects):
    eff = 0
    for key, value in data2.items():
        if key[i]:
            eff += value
        else:
            eff -= value
    eff /= 2**power 
    print(f"Effect({e}): {eff}")

Effect(P): -1.25
Effect(S): 1.75
Effect(N): 0.75
Effect(PS): 0.25
Effect(PN): 2.25
Effect(SN): 2.25
Effect(PSN): -0.25


**Answer to question 5.1(b):** effects are shown above,

the new design matrix is:

|$P$  | $S$ | $N$ | $PS$ | $PN$ | $SN$ | $PSN$ | **Growth rate**  |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|$+$ | $-$ | $-$ | $-$ | $-$ | $+$ | $+$ | $7$ |
|$-$ | $+$ | $-$ | $-$ | $+$ | $-$ | $+$ | $10$ |
|$+$ | $-$ | $+$ | $-$ | $+$ | $-$ | $-$ | $8$ |
|$-$ | $+$ | $+$ | $-$ | $-$ | $+$ | $-$ | $11$ |
|$-$ | $-$ | $-$ | $+$ | $+$ | $+$ | $-$ | $11$ |
|$+$ | $+$ | $+$ | $+$ | $+$ | $+$ | $+$ | $12$ |
|$+$ | $+$ | $-$ | $+$ | $-$ | $-$ | $-$ | $7$ |
|$-$ | $-$ | $+$ | $+$ | $-$ | $-$ | $+$ | $7$ |

| |
|:--|
|**Table 2:** *Experimental design matrix for the growth rate of the investigated bateria. The factors are the concentration of phosphate ($P$), the concentration of sucrose ($S$) and the concentration of nitrate ($N$).*|


**(c)** What factors and interactions seem
to increase the growth rate?



**Answer to question 5.1(c):** From the output of question **(b)** the things with a positive factor are:
S, N, PS, PN, and SN.

**(d)**  Make two least-squares models of the data
given in table 1:

* (i)  Model 1, which only includes the main effects.

* (ii)  Model 2, which includes the main effects and
the interactions.


When making the least-squares models, convert "$+$" to $1$ and
"$-$" to $-1$.

Compare the two models with the effects you have calculated, and the
conclusions you made in point **(c)**.


In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression

xdata1 = np.empty((8, 3)) # 8 measurements with 3 variables
xdata2 = np.empty((8, 7)) # 8 measurements with all variables

ydata = np.empty((8, 1))

for i, (key, value) in enumerate(data2.items()):
    ydata[i] = value

    x = [1 if j==1 else -1 for j in key]
    xdata1[i] = x[:3]
    xdata2[i] = x

a0 = ydata.mean()  # a0 is to average the data
model1 = LinearRegression()
model1.fit(xdata1, ydata)

model2 = LinearRegression()
model2.fit(xdata2, ydata);

coeff1 = [i for i in model1.coef_[0]]
coeff2 = [i for i in model2.coef_[0]]

out = f"Model 1: {a0}"
for eff, coeff in zip(effects, coeff1):
    out += f" + {coeff:5.3f} * {eff}"
print(out)

out = f"Model 2: {a0}"
for eff, coeff in zip(effects, coeff2):
    out += f" + {coeff:5.3f} * {eff}"
print(out)

Model 1: 9.125 + -0.625 * P + 0.875 * S + 0.375 * N
Model 2: 9.125 + -0.625 * P + 0.875 * S + 0.375 * N + 0.125 * PS + 1.125 * PN + 1.125 * SN + -0.125 * PSN


**Answer to question 5.1(d):** The first thing we note, is that the coefficients have identical signs to their corresponding
effects, for instance, both $a_1$ and
$a_7$ are negative as the effects of $P$ and $PSN$ are. Thus, we again find that
$S$, $N$, $PS$, $PN$ and $SN$ influences the growth rate in a positive way.

Secondly, we note that $a_0$ is just
the average response, $a_0 = \tfrac{1}{8} (7 + 10+8+11+11+12+7+7) = \frac{73}{8}$.

Thirdly, we see that for all coefficients $a_1, \ldots, a_7$ we have that the regression
coefficient is $\tfrac{1}{2}$ of the corresponding effect. We have for instance
that $a_2 = \tfrac{7}{8} = \tfrac{1}{2} \times \tfrac{7}{4} = \tfrac{\text{Effect}(S)}{2}$.
Let see if we can understand this by calculating the effect of $S$,

\begin{equation*}
    \text{Effect}(S) = \tfrac{1}{4} \left( (y_2 + y_4 + y_6 + y_7) -
                        (y_1 + y_3 + y_5 + y_8)\right),
\end{equation*}

using the equation for model 2 for $y_i$ with the corresponding values for
$P$, $S$ and $N$.
It is a bit tedious to write this out,
so let us rewrite the equation as $y = \mathbf{a}^\top \mathbf{x}$, where
$\mathbf{a}^\top = [a_0, a_1, \ldots, a_7]$, and
$\mathbf{x}^\top = [1, P, S, \ldots, PSN]$.
The effect of $S$ is then,

\begin{equation*}
    \begin{split}
    \text{Effect}(S) &= \tfrac{1}{4} \mathbf{a}^\top
                       \left((\mathbf{x}_2 + \mathbf{x}_4 + \mathbf{x}_6 + \mathbf{x}_7) -
                        (\mathbf{x}_1 + \mathbf{x}_3 + \mathbf{x}_5 + \mathbf{x}_8)
                    \right) \\
    &= \tfrac{1}{4} \mathbf{a}^\top [0, 0, 8, 0, 0, 0, 0, 0]^\top \\
    &= \tfrac{8}{4} a_2 = 2 a_2,
    \end{split}
\end{equation*}

and similarly for the other coefficients. That is, the effects are twice the
regression coefficients. More generally, if the "high" value
of $S$ is $s_+$ and the "low" is $s_-$, then we get (by the same approach),

\begin{equation*}
    \text{Effect}(S) = \tfrac{1}{4} \mathbf{a}^\top [0, 0, 4(s_+ - s_-), 0, 0, 0, 0, 0]^\top
                       = (s_+ - s_-) a_2.
\end{equation*}

Thus, the general relation between a regression coefficient and the
corresponding effect is,

\begin{equation*}
    a_i = \frac{
            \text{Effect}(i)
        }{
            i_{+} - i_{-}
        },
\end{equation*}

where $i$ is the factor, $a_i$ the corresponding regression coefficient and
$i_+$ is the value for which $i$ is considered "high" and $i_-$ is the value
for which $i$ is considered "low".

In conclusion, the regression coefficients contain the same information as
the effects we have previously calculated.



**Exercise 5.2**

You have recently started a new job as the lead
experimental chemist in a company that makes chocolate bars.
A new, and supposedly tasty, chocolate is being developed, and the
main ingredients that you can vary are: 


*  The amount of cocoa ($A$).

*  The number of pecan nuts ($B$).

*  The amount of caramel ($C$).

*  The amount of milk powder ($D$).

*  The amount of sugar ($E$).

*  The amount of vanilla ($F$).

You are tasked with carrying out a maximum of $16$ experiments (limited due to cost
and time constraints) in which the best mixture of the main ingredients ($A$–$F$)
is found. ("Best" is here determined by a tasting panel made up of $30$ people.)
For this task you decide on making a two-level fractional factorial design.


**(a)**  How many experiments would you have to carry out
if you were to perform a full factorial design?



In [4]:
print(f"Number of experiments for full factorial design is {2**6}")

Number of experiments for full factorial design is 64


**Answer to question 5.2(a):** See above.

**(b)**  As stated, you can only carry out $16$ experiments. 
Explain what confounding is and why the set up with
$16$ experiments will lead to confounding.



**Answer to question 5.2(b):** 

When we carry out a full factorial design, we do enough
experiments so that we can determine the effect of all
factors. When we
do a fractional factorial design, we do too few experiments
to uniquely determine all the effects. When we estimate effects,
we then find that the expressions for some of the effects are
identical and there is no way that we can distinguish between them.
The extreme case would be if we only carried out one experiment,
then we would not have any information on how the different
variables influences out measured quantity.

In the present case, we need $64$ experiments (see answer at **(a)**)
but we can only do $16$. We therefore know that some of our factors
will be confounded with each other.

Luckily, we have some choice and we can, for instance, make it so
that the main effects are confounded with higher-order interactions.
This is useful as we often can neglect higher-order interactions
compared to the main effects.


**(c)**  After talking with the chocolate design team, you
decide on the following generators:

*  $E = ABC$.

*  $F = BCD$.

What is a defining contrast, and what are the
defining contrasts in this case?



**Answer to question 5.2(c):** 

A defining contrast is a
relation that can be used to deduce
what variables are confounded.
In this case, we decided on two generators which can be used
to generate the defining contrasts,
*  $E = ABC \implies E^2 = 1 = ABCE$.
*  $F = BCD \implies F^2 = 1 = BCDF$.

In addition, we have that,
\begin{equation*}
1 \times 1 = 1 = ABCE \times BCDF = ADEF.
\end{equation*}

Thus, we have $3$ defining constrasts here:
* $1 = ABCE$.
* $1 = BCDF$.
* $1 = ADEF$.


**(d)**  Are any of the main effects confounded with $2$-factor
interactions in this case?



**Answer to question 5.2(d):**

Here, we have made the choices $E = ABC$ and $F = BCD$.
This means that we confound the two main effects $E$ and $F$
with $3$-factor interactions. Let check the confounding in
full detail for the main effects (see also the summary in
Table 3):

* For $A$ we have:
    * $A = A \times 1 = A \times ABCE = BCE$,
    * $A = A \times 1 = A \times BCDF = ABCDF$,
    * $A = A \times 1 = A \times ADEF = DEF$,
  
  |and we see that $A$ is confounded with $BCE$, $DEF$ and
  $ABCDF$.
* For $B$ we have:
  * $B = B \times 1 = B \times ABCE = ACE$,
  * $B = B \times 1 = B \times BCDF = CDF$,
  * $B = B \times 1 = B \times ADEF = ABDEF$,
  
  and we see that $B$ is confounded with $ACE$, $CDF$ and
  $ABDEF$.
* For $C$ we have:
  * $C = C \times 1 = C \times ABCE = ABE$,
  * $C = C \times 1 = C \times BCDF = BDF$,
  * $C = C \times 1 = C \times ADEF = ACDEF$,
  
  and we see that $C$ is confounded with $ABE$, $BDF$ and
  $ACDEF$.
* For $D$ we have:
  * $D = D \times 1 = D \times ABCE = ABCDE$,
  * $D = D \times 1 = D \times BCDF = BCF$,
  * $D = D \times 1 = D \times ADEF = AEF$,
  
  and we see that $D$ is confounded with $AEF$, $BCF$ and
  $ABCDE$.
* For $E$ we have:
  * $E = E \times 1 = E \times ABCE = ABC$,
  * $E = E \times 1 = E \times BCDF = BCDEF$,
  * $E = E \times 1 = E \times ADEF = ADF$,
  
  and we see that $E$ is confounded with $ABC$, $ADF$ and
  $BCDEF$.
* For $F$ we have:
  
  * $F = F \times 1 = F \times ABCE = ABCEF$,
  * $F = F \times 1 = F \times BCDF = BCD$,
  * $F = F \times 1 = F \times ADEF = ADE$,
  
  and we see that $F$ is confounded with $ADE$, $BCD$ and
  $ABCEF$.


Based on this, we conclude that *none* of the main effects are
confounded with $2$-factor interactions.

|**Main effect**  | **Confounded with** |
|:---:|:---:|
|$A$ | $BCE$, $DEF$ & $ABCDF$ |
|$B$ | $ACE$, $CDF$ & $ABDEF$ |
|$C$ | $ABE$, $BDF$ & $ACDEF$ |
|$D$ | $AEF$, $BCF$ & $ABCDE$ |
|$E$ | $ABC$, $ADF$ & $BCDEF$ |
|$F$ | $ADE$, $BCD$ & $ABCEF$ |

| |
|:--|
|**Table 4:** *Confounding between variables in the $2^{6-2}_\text{IV}$ fractional factorial design.*|




**(e)**  What is the resolution for this design? Write out
the short-hand representation of this design on the
form $2^{N-p}_R$, where $N$ is the number of factors,
$p$ is the number of generators, and $R$ the resolution.



**Answer to question 5.2(e):** 

The resolution is identical to the length of the shortest
defining contrast. From point **(c)** we see that
the shortest defining contrast has a length of $4$. This means
that our design is a $2^{6-2}_\text{IV}$ fractional factorial design.

We also know that given the resolution $R$, a $p$-order factor will
be confounded with other factors of order $R-p$ or higher. For the
main effects, $p=1$, and we have then that these effects will be
confounded with factors of order $R-p = 4 - 1 = 3$ or higher. This
is exactly what we found in point **(d)**. Thus, we could
have answered the previous question without all the calculations just
by considering the resolution.


**(f)**  Construct the design matrix for the current design, but show
only the columns for the main effects.



In [5]:
import itertools as itt
a = [0, 1]
b = [0, 1]
c = [0, 1]
d = [0, 1]

print("a  b  c  d  e  f")

for ai, bi, ci, di in itt.product(a,b,c,d):

    # E = abc
    ei = ai == (bi == ci) # Assign the truth of ai == (bi == ci) to ei, not the first single equal

    # F = bcd
    fi = bi == (ci == di)

    out = ["+" if i else "-" for i in [ai, bi, ci, di, ei, fi]]
    print("  ".join(out))

a  b  c  d  e  f
-  -  -  -  -  -
-  -  -  +  -  +
-  -  +  -  +  +
-  -  +  +  +  -
-  +  -  -  +  +
-  +  -  +  +  -
-  +  +  -  -  -
-  +  +  +  -  +
+  -  -  -  +  -
+  -  -  +  +  +
+  -  +  -  -  +
+  -  +  +  -  -
+  +  -  -  -  +
+  +  -  +  -  -
+  +  +  -  +  -
+  +  +  +  +  +


**Answer to question 5.2(f):** 

The design matrix for the $6$ main effects
is given in the prints above and the table below.

|$A$  | $B$ | $C$ | $D$ | $E$ | $F$ |
|:---:|:---:|:---:|:---:|:---:|:---:|
|$-$ | $-$ | $-$ | $-$ | $-$ | $-$ |
|$-$ | $-$ | $-$ | $+$ | $-$ | $+$ |
|$-$ | $-$ | $+$ | $-$ | $+$ | $+$ |
|$-$ | $-$ | $+$ | $+$ | $+$ | $-$ |
|$-$ | $+$ | $-$ | $-$ | $+$ | $+$ |
|$-$ | $+$ | $-$ | $+$ | $+$ | $-$ |
|$-$ | $+$ | $+$ | $-$ | $-$ | $-$ |
|$-$ | $+$ | $+$ | $+$ | $-$ | $+$ |
|$+$ | $-$ | $-$ | $-$ | $+$ | $-$ |
|$+$ | $-$ | $-$ | $+$ | $+$ | $+$ |
|$+$ | $-$ | $+$ | $-$ | $-$ | $+$ |
|$+$ | $-$ | $+$ | $+$ | $-$ | $-$ |
|$+$ | $+$ | $-$ | $-$ | $-$ | $+$ |
|$+$ | $+$ | $-$ | $+$ | $-$ | $-$ |
|$+$ | $+$ | $+$ | $-$ | $+$ | $-$ |
|$+$ | $+$ | $+$ | $+$ | $+$ | $+$ |

| |
|:--|
|**Table 5:** *Experimental design matrix for the $2^{6-2}_\text{IV}$ fractional factorial design.*|


**(g)**  Another member of your team suggests doing just $8$ experiments
as this will cut time and cost.
Do you think this is a good idea? Why/why not? What would
the design matrix look like in this case?


In [6]:
a = [0, 1]
b = [0, 1]
c = [0, 1]

print("a  b  c  d  e  f")

for ai, bi, ci in itt.product(a,b,c):

    # D = AB
    di = ai == bi

    # E = AC
    ei = ai == ci

    # F = bc
    fi = bi == ci

    out = ["+" if i else "-" for i in [ai, bi, ci, di, ei, fi]]
    print("  ".join(out))

a  b  c  d  e  f
-  -  -  +  +  +
-  -  +  +  -  -
-  +  -  -  +  -
-  +  +  -  -  +
+  -  -  -  -  +
+  -  +  -  +  -
+  +  -  +  -  -
+  +  +  +  +  +


**Answer to question 5.2(g):**

With $8$ experiments, we get even less information and we
expect the confounding to change. This is a $2^{6-3}$
fractional design. In this case we can determine $3$ of the
main effects fully ($2^3 = 2^{6-3}$) and for the remaining $3$
we have to select generators. Let us say that we select $A$,
$B$ and $C$ as the main effects we determine fully. Then,
we have to choose generators for $D$, $E$ and $F$ by combining
$A$, $B$ and $C$. We can make $3$ combinations using $2$ of
the three variables (the general formula
for the number of combinations when we have $n$ objects and select $r$ of them is
$n!/(r!(n-r)!)$), for instance,


using $2$ of
the three variables, for instance,

* (i)  $D = AB$,

* (ii)  $E = AC$,

* (iii)  $F = BC$.


The defining contrasts will then be,

* (i)  $D = AB \implies D^2 = 1 = ABD$,

* (ii)  $E = AC \implies E^2 = 1 = ACE$,

* (iii)  $F = BC \implies F^2 = 1 = BCF$,

* (iv)  $1 = 1 \times 1 = ABD \times ACE = BCDE$,

* (v)  $1 = 1 \times 1 = ABD \times BCF = ACDF$,

* (vi)  $1 = 1 \times 1 = ACE \times BCF = ACDF$,

* (vii)  $1 = 1 \times 1 \times 1 = ABD \times ACE \times BCF = DEF$.


We see from this that the shortest defining contrast has a length
of $3$ (i.e. the resolution is $R=3$) and we have
a $2^{6-3}_{\text{III}}$ fractional factorial design.
We know then that the main effects ($p=1$) will be
confounded with factors of order $R-p = 3 - 1 = 2$ or higher.
So we have that our main effects will be confounded with $2$-factor
interactions.

In general, we want our main effects to not be confounded with
interactions lower than $3$-factor interactions. We would then
advise against doing only $8$ experiments, and keep to the
original plan with $16$ experiments to be able to better
assess the main effects.

The design matrix for our selection of generators is
given in the table below and the print above. Note that you might
get different design matrices, depending on your choice of
generators.

|$A$  | $B$ | $C$ | $D$ | $E$ | $F$ |
|:---:|:---:|:---:|:---:|:---:|:---:|
|$-$ | $-$ | $-$ | $+$ | $+$ | $+$ |
|$-$ | $-$ | $+$ | $+$ | $-$ | $-$ |
|$-$ | $+$ | $-$ | $-$ | $+$ | $-$ |
|$-$ | $+$ | $+$ | $-$ | $-$ | $+$ |
|$+$ | $-$ | $-$ | $-$ | $-$ | $+$ |
|$+$ | $-$ | $+$ | $-$ | $+$ | $-$ |
|$+$ | $+$ | $-$ | $+$ | $-$ | $-$ |
|$+$ | $+$ | $+$ | $+$ | $+$ | $+$ |

| |
|:---|
|**Table 6:** *Experimental design matrix for the $2^{6-3}_\text{III}$ fractional factorial design.*|